### OSINT Goal of Version 2.0：
<B>Main Issues</B>
1. Sorting: According to search topic, push those highly repeatability back, put those highly related in front
2. Information Extraction: According to search topic
3. Relation Extraction: create knowledge graph

--------------------------------------------------------
<B>Others</B>
1. Model Selection: determine to use LAC or CKIP
    - LAC: 由簡體中文語料訓練而得
    - CKIP: 由繁體中文語料訓練而得

Chinese Corpus Github: https://github.com/InsaneLife/ChineseNLPCorpus

## 使用192.168.70.4主機跑（因較耗時）

In [22]:
import os
os.getcwd() # 返回目前所在路徑

'/home/jovyan/OSINT/code'

In [23]:
import math
import pickle
import pandas as pd
import numpy as np
import re
from LAC import LAC
from zhon.hanzi import punctuation as zhPunc
from string import punctuation as enPunc
from collections import Counter

In [24]:
main_dir = '/home/jovyan/OSINT'
file_dir = 'Corpus/for_text_classification'

### Process THUCTC text classification corpus 中文文本數據集

THUCNews是根据新浪新闻RSS订阅频道2005~2011年间的历史数据筛选过滤生成，包含74万篇新闻文档（2.19 GB），均为UTF-8纯文本格式。我们在原始新浪新闻分类体系的基础上，重新整合划分出14个候选分类类别：财经、彩票、房产、股票、家居、教育、科技、社会、时尚、时政、体育、星座、游戏、娱乐。使用THUCTC工具包在此数据集上进行评测，准确率可以达到88.6%。

http://thuctc.thunlp.org/#中文文本分类数据集THUCNews

<B>Download Dataset</B>
- wget https://thunlp.oss-cn-qingdao.aliyuncs.com/THUCNews.zip
- unzip THUCNews.zip

In [4]:
classes = os.listdir(f'{main_dir}/{file_dir}/THUCNews')  # THUCTC corpus includes several classes
classes

['财经',
 '教育',
 '社会',
 '时政',
 '科技',
 '游戏',
 '体育',
 '时尚',
 '娱乐',
 '家居',
 '彩票',
 '房产',
 '星座',
 '股票']

In [28]:
# 查詢特定檔案資料
text = pd.read_csv(f'{main_dir}/{file_dir}/THUCNews/股票/751240.txt', header=None, delimiter='\t\n\0')
text

nan
nan
nan


### 資料處理：調整為LAC model input 格式

In [25]:
# 文本斷句
temp = [l for i in text[0] if type(i) == str for l in re.split(r'[，,。：:；;！!？?]',i) if l != '']
# temp = []
# for i in text[0]:
#     if type(i) == str:
#         for l in re.split(r'[，,。：:；;！!？?]',i):
#             if l != '':
#                 temp.append(l)
temp

['2月24日上市公司周末晚间公告速递',
 '上海证券交易所上市公司',
 '根据有关文件的规定',
 '长信基金管理有限责任公司定于2009年2月27日开始办理长信利丰债券型证券投资基金(简称',
 '长信利丰',
 '代码',
 '519989',
 '下称',
 '本基金)的基金份额日常赎回业务',
 '现就本基金有关赎回业务具体事宜公告如下',
 '本基金开放日为上海证券交易所、深圳证券交易所的正常交易日',
 '具体业务办理时间为上述两交易所的交易时间',
 '场外赎回时',
 '每次对本基金的赎回申请不得低于1.00份基金份额',
 '场内赎回时',
 '赎回的最低份额为1.00份',
 '基金份额持有人赎回时或赎回后',
 '导致每个交易帐户的基金份额余额不足1.00份时',
 '在赎回时需一次全部赎回',
 '赎回费率为0.1%(不计年限)',
 '海通食品集团股份有限公司于2009年2月23日收到股东叶成浩有关函',
 '叶成浩自2009年2月12日起至2月20日止累计出售公司流通股78328股(占公司总股本的0.03%)',
 '目前持有公司流通股11458946股(占公司总股本的4.98%)',
 '江苏江南高纤股份有限公司于2009年2月19日接到发起人股东上海鑫荻良实业发展有限公司[股权分置改革后持有公司4213907股限售流通股(已取得上市流通权)',
 '占公司总股本的1.95%',
 '下称',
 '鑫荻良]的通知',
 '截至2008年1月15日',
 '鑫荻良已累计出售公司股份2224889股(占公司总股本的0.95%)',
 '2008年5月公司资本公积金转增股本后',
 '鑫荻良还持有公司股票2983527股(占公司股份总数的0.85%)',
 '自2009年2月13日至16日出售完毕',
 '云南驰宏锌锗股份有限公司近日接到2008年度审计机构亚太中汇会计师事务所有限公司(下称',
 '亚太中汇)有关函',
 '亚太中汇与中审会计师事务所有限公司总所及其湖北、上海、广东分所合并',
 '并从2009年2月9日开始正式更名为“中审亚太会计师事务所有限公司”',
 '相关工商变更登记手续已办理完毕',
 '中兵光电科技股份有限公司于2009年2月23日以通讯表决方式召开三届三十次董事会',
 '会议审议同意公司为控股90.69

### LAC分詞

In [25]:
STOPWORD_PATH = f'{main_dir}/Corpus/baidu_stopwords.txt'
__stopwords = list("\xa0\r\n\t\f\v□ ") + list(enPunc) + list(zhPunc)
# __converter = OpenCC("tw2sp")

with open(STOPWORD_PATH, "r") as f:
    lines = f.read().splitlines()
    __stopwords.extend(lines)
    for line in lines:
        __stopwords.append(line)#__converter.convert(line))

In [14]:
# test = text[0][3].split('，')
lac = LAC(mode='seg')
word_seg = lac.run(temp)

W0118 09:05:36.366871    39 analysis_predictor.cc:1353] Deprecated. Please use CreatePredictor instead.


In [20]:
# 單篇文章分詞集合
file_seg = [sentence_seg[idx] for sentence_seg in word_seg for idx in range(len(sentence_seg))]
file_seg

['惠普',
 '或',
 '将',
 '推出',
 '无线',
 '支付',
 '移动',
 '设备',
 '新浪科技讯',
 ' ',
 '北京',
 '时间',
 '6月11日',
 '早间',
 '消息',
 '据',
 '消息',
 '人士',
 '周五',
 '透露',
 '惠普',
 '目前',
 '正在',
 '开发',
 '能够',
 '让',
 '消费者',
 '进行',
 '无线',
 '支付',
 '购物',
 '的',
 '移动',
 '设备',
 '据',
 '该',
 '消息',
 '人士',
 '指出',
 '惠普',
 '正',
 '计划',
 '推出',
 '基于',
 '近场',
 '通信',
 '(',
 'near',
 ' ',
 'field',
 ' ',
 'communication',
 '以下',
 '简称',
 'NFC',
 ')',
 '技术',
 '的',
 '手机',
 '以及',
 '平板',
 '电脑',
 '产品',
 'NFC',
 '技术',
 '能够',
 '让',
 '用户',
 '只需',
 '将',
 '移动',
 '设备',
 '靠近',
 '刷卡器',
 '或者',
 '超市',
 '里',
 '的',
 '收款机',
 '便',
 '可',
 '完成',
 '付款',
 '、',
 '下载',
 '优惠券',
 '或者',
 '获得',
 '忠诚度',
 '积分',
 '等',
 '活动',
 '目前',
 '惠普',
 '尚未',
 '公布',
 '这',
 '一',
 '计划',
 '但是',
 '据',
 '消息',
 '人士',
 '透露',
 '第一款',
 'NFC',
 '产品',
 '将会',
 '在',
 '今年',
 '年底',
 '上市',
 '惠普',
 '在',
 '2010年',
 '以',
 '12亿美元',
 '的',
 '价格',
 '收购',
 '了',
 '智能',
 '手机',
 '生产商',
 'Palm',
 '并',
 '不断',
 '为',
 '这',
 '一',
 '逐渐',
 '失去',
 '智能',
 '手机',
 '市场',
 '份额',
 '的',
 '公司',
 '推出',
 '新',
 '的',
 '功能',
 '以

### 批量處理文件：每個類別的每份文件做分詞，儲存為List，pickle檔
data存放於Corpus/for_text_classification/word_segments

In [ ]:
classes = os.listdir(f'{main_dir}/{file_dir}/THUCNews')
lac = LAC(mode='seg')
for cls in classes:
    cls_list = []
    eachfile = os.listdir(f'{main_dir}/{file_dir}/THUCNews/{cls}')
    print(cls)
    for eachtext in eachfile:
        print(eachtext)
        file_path = f'{main_dir}/{file_dir}/THUCNews/{cls}/{eachtext}'
        if os.path.getsize(file_path) > 10:
            text = pd.read_csv(file_path, header=None, delimiter='\t\n\0')
            if text.empty:
                print('class: %s ; file name: %s' %(cls, eachtext))
                pass
            else:
                text_seg = [phrases for sentence in text[0] for phrases in re.split(r'[，,。：:；;！!？?]',sentence) if phrases != '']
                word_seg = lac.run(text_seg)
                file_seg = [sentence_seg[idx] for sentence_seg in word_seg for idx in range(len(sentence_seg))]
                new_seg = [each_word_seg for each_word_seg in file_seg if each_word_seg not in __stopwords]
        cls_list.append(new_seg)
#     print(cls_list)
    with open(f'{main_dir}/{file_dir}/word_segments/{cls}.pickle', 'wb') as f:
        pickle.dump(cls_list, f)
    print('Finished segment class %s'%cls)

### Debug

In [ ]:
# 股票/751240.txt 資料型態有float
for i in range(len(text[0])):
    if type(text[0][i]) != str:
        print(str(text[0][i]))

In [59]:
# 体育/1568.txt 內容為空，但size = 6
eachfile = os.listdir(f'{main_dir}/{file_dir}/THUCNews/体育')
for eachtext in eachfile:
    file_path = f'{main_dir}/{file_dir}/THUCNews/体育/{eachtext}'
    print(eachtext, os.path.getsize(file_path))

101593.txt 2620
41445.txt 2794
105765.txt 3654
105196.txt 1377
119747.txt 3564
17292.txt 1060
26417.txt 3439
18411.txt 3161
112119.txt 4037
3635.txt 1869
45006.txt 3824
114521.txt 5091
77074.txt 3199
8361.txt 1375
39305.txt 1694
14235.txt 5107
56333.txt 2098
103704.txt 2718
44845.txt 3586
89784.txt 5052
86320.txt 3062
96661.txt 1803
101669.txt 2549
4282.txt 2575
67721.txt 4956
31817.txt 3567
108914.txt 358
114635.txt 3592
93228.txt 2853
93280.txt 2857
30169.txt 3185
6273.txt 2622
42649.txt 1631
6955.txt 2284
116395.txt 1286
153.txt 2662
48850.txt 2688
23529.txt 13559
90688.txt 2831
10511.txt 4980
131143.txt 1700
52183.txt 1319
37826.txt 2839
123458.txt 1635
12709.txt 3048
46312.txt 841
473.txt 3138
75655.txt 6028
91738.txt 1650
111230.txt 3063
44413.txt 1874
35376.txt 4119
43837.txt 995
8486.txt 2076
106975.txt 3642
19613.txt 2424
116125.txt 1811
126400.txt 8712
70384.txt 2018
107811.txt 2573
97429.txt 3513
33273.txt 4103
11372.txt 1653
16376.txt 3786
9106.txt 3384
56677.txt 4666
11239

50649.txt 4136
16153.txt 1415
2510.txt 2633
36545.txt 1016
43110.txt 2003
129635.txt 2184
64303.txt 3742
32865.txt 3083
17688.txt 1608
93524.txt 2719
115881.txt 3146
100391.txt 2377
40439.txt 2591
123359.txt 5495
3118.txt 2678
23752.txt 1468
32402.txt 2851
95360.txt 2215
94783.txt 4109
109178.txt 310
18176.txt 1209
67153.txt 2671
19639.txt 3545
12677.txt 1546
53702.txt 892
59655.txt 5720
38006.txt 7162
111648.txt 3654
46727.txt 13373
96131.txt 267
83093.txt 2332
4100.txt 2121
11481.txt 2945
28129.txt 2319
55007.txt 3469
95893.txt 2086
3541.txt 2227
30407.txt 3047
20354.txt 1598
108733.txt 9153
22703.txt 3496
103561.txt 2234
55087.txt 3177
34836.txt 1147
107345.txt 3074
72418.txt 7457
26524.txt 6572
80706.txt 2728
56579.txt 2938
57948.txt 4278
2233.txt 2919
92816.txt 2262
82632.txt 3177
77060.txt 5749
17665.txt 6224
86304.txt 3531
108118.txt 2560
110179.txt 1005
48893.txt 1359
110025.txt 6776
123624.txt 4346
85415.txt 1747
61550.txt 1677
97232.txt 999
9535.txt 2576
56996.txt 1303
15982.

82628.txt 284
91997.txt 1898
101902.txt 4179
28862.txt 3788
54534.txt 2347
3448.txt 3243
17109.txt 1731
81054.txt 3065
49460.txt 1782
120479.txt 5910
36693.txt 934
17714.txt 1847
10963.txt 1575
41762.txt 4058
7261.txt 437
93998.txt 2744
66841.txt 3402
111777.txt 3245
79137.txt 4368
125692.txt 3752
19409.txt 1734
105329.txt 4634
113125.txt 5816
56334.txt 2970
38102.txt 1495
81808.txt 1745
52786.txt 2317
110992.txt 605
89467.txt 1753
102954.txt 2546
101991.txt 2965
128533.txt 7406
48365.txt 1037
114994.txt 1413
61113.txt 5995
34338.txt 720
15285.txt 1260
89158.txt 2095
49433.txt 3156
75224.txt 2218
52658.txt 1807
74435.txt 1507
68714.txt 1735
77070.txt 3801
27635.txt 2222
116321.txt 2714
4628.txt 2240
64575.txt 548
88317.txt 2915
7819.txt 3370
65150.txt 511
45368.txt 3606
63669.txt 920
6042.txt 1760
66658.txt 1211
78492.txt 1897
55266.txt 3500
4243.txt 7692
80901.txt 3414
79330.txt 3830
65110.txt 366
108257.txt 3112
66952.txt 4473
62797.txt 1910
41866.txt 3445
57106.txt 2425
102679.txt 2

736.txt 3133
34081.txt 3244
42688.txt 1969
123747.txt 5549
21774.txt 2115
51301.txt 3239
124832.txt 3501
125321.txt 3036
13676.txt 1480
128405.txt 3023
19949.txt 1015
122483.txt 1091
106970.txt 2592
78083.txt 385
86313.txt 3217
38158.txt 3223
120012.txt 5675
130946.txt 1477
25792.txt 8211
52570.txt 4816
121369.txt 1632
4220.txt 1355
88898.txt 3954
86997.txt 419
89415.txt 2745
32275.txt 1328
39078.txt 2121
98387.txt 2488
111254.txt 2789
50027.txt 1279
114380.txt 637
51544.txt 824
28042.txt 1844
101494.txt 3017
57101.txt 4722
43538.txt 2531
43204.txt 5441
38682.txt 643
80015.txt 1018
87380.txt 3699
121089.txt 1593
62133.txt 7165
63491.txt 5511
74947.txt 3997
125587.txt 13531
50929.txt 1247
50185.txt 3542
86078.txt 3121
93616.txt 2304
68437.txt 3621
902.txt 16763
127120.txt 5474
93860.txt 2725
61007.txt 3345
3245.txt 3147
90159.txt 2946
41818.txt 952
13141.txt 294
2515.txt 423
21974.txt 2870
21695.txt 5156
3664.txt 2615
53787.txt 1319
15180.txt 1273
66499.txt 559
1496.txt 2164
24967.txt 3

62840.txt 2714
64082.txt 2739
93215.txt 3824
48075.txt 3915
93808.txt 1371
96226.txt 5125
10557.txt 2707
14674.txt 2560
1163.txt 18733
10650.txt 1239
46733.txt 7340
130227.txt 956
109681.txt 197
12129.txt 1989
121999.txt 6751
30809.txt 2099
79613.txt 694
85864.txt 1900
80503.txt 3499
110107.txt 10962
97000.txt 1279
80276.txt 5034
51999.txt 2801
15444.txt 1276
36076.txt 1239
16238.txt 524
16335.txt 560
64076.txt 2392
54231.txt 405
76051.txt 3668
121846.txt 1319
20603.txt 4878
8964.txt 3514
97667.txt 3190
54492.txt 8225
13672.txt 1926
8856.txt 1370
84452.txt 3926
126069.txt 2656
105695.txt 2532
126164.txt 2015
67003.txt 2710
5072.txt 2970
117626.txt 2112
90890.txt 2636
88149.txt 1762
38969.txt 4529
53791.txt 2965
112466.txt 5743
130608.txt 6208
95828.txt 6918
44732.txt 1697
75034.txt 1521
6839.txt 2720
116466.txt 1359
20044.txt 7047
39043.txt 708
34307.txt 1194
33999.txt 1134
77051.txt 11932
71452.txt 485
43729.txt 962
89580.txt 3370
53520.txt 1576
93884.txt 2713
94247.txt 2127
85682.txt

...

44728.txt 4172
123692.txt 1325
91979.txt 3536
123166.txt 5838
88286.txt 2264
13015.txt 236
16367.txt 4551
696.txt 2009
127705.txt 3173
56009.txt 2528
109730.txt 1651
118649.txt 4339
112880.txt 1153
26631.txt 3170
3973.txt 3373
35361.txt 3110
70385.txt 4068
131026.txt 3478
102521.txt 1872
42289.txt 1141
113171.txt 3048
62409.txt 4146
95184.txt 2724
2182.txt 2830
74259.txt 2422
106876.txt 3244
10527.txt 2904
35785.txt 4633
78865.txt 3069
115686.txt 3836
98889.txt 3126
71489.txt 7915
28983.txt 6091
75953.txt 2045
47954.txt 2847
86058.txt 2236
66967.txt 4916
92277.txt 3368
99378.txt 3175
28547.txt 1742
59534.txt 4842
4411.txt 1796
8318.txt 2393
11850.txt 2511
17685.txt 3299
6294.txt 1531
75339.txt 761
14498.txt 2969
1894.txt 3184
59051.txt 2481
117715.txt 2872
56403.txt 2162
92801.txt 12360
5078.txt 3873
110066.txt 3485
65056.txt 820
13236.txt 1283
81548.txt 2437
23855.txt 1702
67628.txt 1775
117801.txt 2163
103466.txt 2407
33443.txt 4083
129203.txt 2639
116984.txt 1597
88518.txt 2093
5025

130219.txt 6800
97771.txt 1389
22821.txt 2566
80802.txt 2253
17568.txt 416
8861.txt 2220
12253.txt 2811
48287.txt 8802
45719.txt 3758
34509.txt 4106
127372.txt 1806
64382.txt 637
74826.txt 3965
18118.txt 2859
109696.txt 179
96768.txt 2872
70350.txt 2993
84699.txt 6292
22521.txt 1614
124965.txt 6719
74927.txt 5275
115903.txt 3266
124426.txt 2471
91332.txt 2393
13166.txt 342
95863.txt 2478
123915.txt 1761
19770.txt 1268
94155.txt 3026
64492.txt 738
33050.txt 1080
1801.txt 1057
85980.txt 3347
109582.txt 276
61399.txt 3839
82818.txt 1825
81754.txt 2259
70691.txt 2287
18918.txt 396
35289.txt 4042
46563.txt 4064
107465.txt 1234
22630.txt 3047
65517.txt 521
27101.txt 5321
80024.txt 128
82527.txt 2580
3421.txt 7398
60524.txt 1575
104036.txt 4365
102373.txt 2432
121474.txt 4317
58159.txt 3026
124681.txt 3449
129829.txt 6914
64293.txt 3203
17067.txt 1812
5938.txt 1391
31908.txt 3842
111794.txt 2426
20972.txt 1485
87363.txt 3249
49687.txt 2252
85343.txt 1297
101539.txt 1975
72449.txt 2841
10105.t

31075.txt 4096
80323.txt 2669
124143.txt 2560
17374.txt 686
13838.txt 1977
127473.txt 8149
6575.txt 1972
113549.txt 1838
18990.txt 450
31582.txt 1415
32668.txt 3007
75056.txt 3159
57763.txt 946
96349.txt 3001
112563.txt 1227
122535.txt 1892
13251.txt 3505
97487.txt 3139
105605.txt 3202
93097.txt 2811
26403.txt 6236
47532.txt 5965
114143.txt 1323
64401.txt 486
38366.txt 4400
95900.txt 1168
116606.txt 2634
8431.txt 3711
32703.txt 4421
20418.txt 2465
81780.txt 5015
59611.txt 23403
18128.txt 2057
64458.txt 448
76679.txt 3723
88075.txt 3144
44370.txt 2451
102836.txt 1834
87853.txt 2359
34861.txt 3720
55452.txt 1823
30240.txt 3438
77341.txt 1921
58055.txt 995
106522.txt 2303
130065.txt 1491
98063.txt 5618
65404.txt 684
31492.txt 3528
67341.txt 3657
111451.txt 3358
65977.txt 400
123763.txt 5986
111282.txt 1514
8788.txt 3397
18899.txt 359
23040.txt 2566
74037.txt 2554
72563.txt 1988
6279.txt 1411
94974.txt 623
106901.txt 2590
38122.txt 11684
40382.txt 1587
111912.txt 1131
59481.txt 3867
58138.

77007.txt 2066
125125.txt 2106
24539.txt 1763
83772.txt 3291
64356.txt 663
49366.txt 4877
45895.txt 834
117413.txt 1408
56112.txt 4577
71240.txt 7561
116658.txt 4877
120886.txt 7554
39673.txt 1484
50339.txt 7184
17559.txt 1468
30359.txt 2554
75724.txt 3939
38715.txt 415
102927.txt 2777
5576.txt 489
37468.txt 1000
64232.txt 1449
91751.txt 1146
123197.txt 2756
40581.txt 2152
56410.txt 2623
73747.txt 2326
87975.txt 2619
75767.txt 2555
73457.txt 3417
121897.txt 1485
59707.txt 824
74889.txt 7587
98597.txt 6942
99246.txt 1523
41611.txt 2096
124408.txt 4587
50494.txt 6087
53449.txt 2265
82062.txt 3368
93889.txt 125
86437.txt 2257
49743.txt 1422
101481.txt 2990
53621.txt 6978
411.txt 2619
22593.txt 3399
69807.txt 6887
100630.txt 9428
122782.txt 3310
109533.txt 733
13134.txt 294
18971.txt 441
32411.txt 3173
95664.txt 2236
78262.txt 4245
118139.txt 757
96509.txt 1593
998.txt 1407
52381.txt 1383
69474.txt 4263
69533.txt 2103
16279.txt 1104
86530.txt 4345
57460.txt 1669
21843.txt 10226
27989.txt 2

103328.txt 2101
67004.txt 967
19301.txt 472
76436.txt 3016
91348.txt 4438
82171.txt 3613
33636.txt 4198
37951.txt 2065
101557.txt 2028
61899.txt 2224
9315.txt 965
112210.txt 1487
41814.txt 848
103152.txt 2193
75564.txt 4201
122055.txt 822
84537.txt 2678
104970.txt 1566
20146.txt 3374
103076.txt 4117
41111.txt 1198
68385.txt 2898
60489.txt 3998
31474.txt 4379
36903.txt 2141
48170.txt 2849
103634.txt 1607
7969.txt 5250
57319.txt 2308
28463.txt 2269
25736.txt 2909
58977.txt 876
9001.txt 8762
71113.txt 531
29452.txt 3856
114353.txt 5826
99597.txt 3292
41544.txt 3452
87327.txt 717
36504.txt 3678
73743.txt 4183
84465.txt 1706
104541.txt 3562
48385.txt 1889
102184.txt 4733
128635.txt 2503
111095.txt 3355
45656.txt 10707
113619.txt 5143
18645.txt 2121
7934.txt 3721
119832.txt 3693
95891.txt 2045
10084.txt 1611
19669.txt 1496
16278.txt 2031
7711.txt 3523
84647.txt 2776
94098.txt 1483
19313.txt 307
24526.txt 2355
61133.txt 3333
30891.txt 1265
87396.txt 2782
112410.txt 4616
37698.txt 3569
74643.t

### 匯入檔案查看

In [109]:
with open(f'{main_dir}/{file_dir}/word_segments/星座.pickle', 'rb') as f:
    loaded_data = pickle.load(f)
print('Loaded data length: ',len(loaded_data))

# 清除\xa0
data = []
for lists in loaded_data:
    temp = []
    for words in lists:
        temp.append(words.replace('\xa0', ''))
    data.append(temp)
print('Used data length: ', len(data))

Loaded data length:  3578
Used data length:  3578


In [111]:
# check if the list data correct
for eachlist in data[:5]:
    print(eachlist[:10])

['个性', '测试', '几分', '童心未泯', '图', '独家', '撰稿', '小周', '心理', '测试']
['个性', '测试', '无条件', '付出', '图', '独家', '撰稿', '小周', '心理', '测试']
['婚后', '最', '宠爱', '老公', '星座', '图', '说', '老公', '爱', '老婆']
['处女座', '最', '想', '收到', '中秋', '礼物', '图', '处女座', '毛绒', '小白兔']
['双鱼座', '爱情', '自我', '解析', '图', '新浪', '网友', '天降神缘', '情感', '故事']


### 各類文本關鍵字萃取

In [117]:
# 計算詞頻
all_doc_cnt = [Counter(eachlist) for eachlist in data]
print(all_doc_cnt[4])

Counter({'小鱼': 46, '去': 23, '不': 23, '事情': 18, '都': 11, '关心': 11, '最': 11, '爱': 11, '做': 9, '喜欢': 9, '完美': 9, '幸福': 8, '双鱼': 8, '想': 7, '对方': 7, '请': 7, '真': 6, '鱼': 6, '人': 6, '一个': 6, '开心': 6, '告诉你': 6, '出': 6, '回忆': 6, '越': 6, '双鱼座': 5, '星座': 5, '照顾': 5, '第一时间': 5, '说': 5, '情侣': 5, '话': 5, '肯定': 5, '爱情': 4, '都是': 4, '回家': 4, '晚': 4, '短信': 4, '想法': 4, '背叛': 4, '浪漫': 4, '女朋友': 4, '上': 3, '一只': 3, '感情': 3, '更': 3, '在一起': 3, '难受': 3, '东西': 3, '想到': 3, '感觉到': 3, '时': 3, '好': 3, '希望': 3, '永远': 3, '担心': 3, '感觉': 3, '怕': 3, '乱七八糟': 3, '一声': 3, '变': 3, '承受': 3, '别人': 3, '千万': 3, '忍受': 3, '情人': 3, '朋友': 3, '不一样': 3, '女': 3, '逼': 3, '新浪': 2, '挺': 2, '好累': 2, '那种': 2, '自然': 2, '头': 2, '靠着': 2, '窗户': 2, '掂': 2, '下': 2, '再': 2, '多喝': 2, '点': 2, '热水': 2, '小事': 2, '都会': 2, '谈恋爱': 2, '心': 2, '碰到': 2, '和你': 2, '送给': 2, '电话': 2, '尽量': 2, '有事': 2, '干': 2, '回': 2, '反正': 2, '不太': 2, '告诉他': 2, '想不到': 2, '花心': 2, '一个人': 2, '付出': 2, '就算': 2, '感到': 2, '看': 2, '改变': 2, '快': 2, '你身边': 2, '溜走': 2, '听': 2, '机会':

In [5]:
def __get_tfidfs(counters, top):
    """取得各篇文章的關鍵字及其透過TF-IDF加權分數的前20名
    
    :param counters: 各篇文章詞頻
    :param top: 前?名
    :return 各篇文章的前?名關鍵詞(dataframe)
    """
    def tf(word, counter):
        return counter[word] / sum(counter.values())

    def idf(word, counters):
        return math.log2(len(counters) / sum(1 for counter in counters if word in counter))

    data = []
    for idx, counter in enumerate(counters):
        data.extend([idx, word, round(score, 5)] for word, score in sorted([
            (word, tf(word, counter) * idf(word, counters)) for word in counter
        ], key=lambda x: x[1], reverse=True)[:top])
    
    return pd.DataFrame(data, columns=["idx", "word", "score"])

In [132]:
tfidf = list(__get_tfidfs(all_doc_cnt[:5], 10)['word'])
print(tfidf)

['测试', '心理', '几分', '童心未泯', '每个孩子', '一夜', '长大', '大人', '返老还童', '人生', '付出', '测试', '心理', '无条件', '一开始', '收获', '就像', '父母', '小时候', '学习', '老公', '婚后', '白羊座', '婚姻', '双子座', '巨蟹座', '中', '爱', '老婆', '家', '处女座', '中秋', '科技', '美丽', '测', '送', '收到', '礼物', '毛绒', '小白兔', '小鱼', '去', '事情', '完美', '双鱼', '对方', '请', '关心', '爱', '鱼']


### 批量處理 （因非常耗時，轉成py檔上傳到70.4，背景執行）

In [7]:
classes = os.listdir(f'{main_dir}/{file_dir}/THUCNews')#[:2]
for cls in classes:
    with open(f'{main_dir}/{file_dir}/word_segments/{cls}.pickle', 'rb') as f:
        loaded_data = pickle.load(f)
    print('Data Type: ', cls)
    print('Loaded data length: ',len(loaded_data))
    
    data = []
    for lists in loaded_data:
        temp = []
        for words in lists:
            temp.append(words.replace('\xa0', ''))
        data.append(temp)
    print('Used data length: ', len(data))
    all_doc_cnt = [Counter(eachlist) for eachlist in data] # 計算詞頻
    tfidf = list(__get_tfidfs(all_doc_cnt, 10)['word'])
    with open(f'{main_dir}/{file_dir}/topic_words/{cls}.pickle', 'wb') as f:
        pickle.dump(tfidf, f)
    print('Finished extract topic words from %s'%cls)
    print('----------------------------------------')

Data Type:  财经
Loaded data length:  37098
Used data length:  5
Finished extract topic words from 财经
----------------------------------------
Data Type:  教育
Loaded data length:  41936
Used data length:  5
Finished extract topic words from 教育
----------------------------------------


In [18]:
# 查看結果
with open(f'{main_dir}/{file_dir}/word_segments/财经.pickle', 'rb') as f:
    loaded_data = pickle.load(f)
print(len(loaded_data))

37098


### OSINT Sorting Method:
<B>Goal : According to search topic, push those highly repeatability back, put those highly related in front</B>

<B>Steps:</B>
1. User gives the search words (and topic ?)
2. Add weight to the keyword of that topic
3. Calculate TF-IDF and get keywords
4. Document similarity

<B>Problems:</B>
1. 現行系統雖有提供標記主題詞彙，但應該量體過小
2. 是否有現成資料？提供已分類的詞彙
3. wiki中文語料能否做分類？